Movie Recommendation System

Importing the dependencies

In [40]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Pre-Processing

In [41]:
#loading the data from the csv file to pandas dataframe
movies_data= pd.read_csv('movies.csv')

In [42]:
#printing the first 5 rows of the dataset
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [43]:
# number of rows and columns in the dataset
movies_data.shape

(4803, 24)

In [44]:
# selecting the relevant features for recommendation
selected_features= ['genres','keywords','tagline', 'cast', 'director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [45]:
# replacing the null values with null string

for feature in selected_features:
    movies_data[feature]= movies_data[feature].fillna('')

In [46]:
# combinig all the selected features

combined_features= movies_data['genres']+movies_data['keywords']+ movies_data['tagline']+ movies_data['cast'] + movies_data['director']
combined_features

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

In [47]:
# converting the textual data to feature vectors

vectorizer= TfidfVectorizer()
feature_vectors= vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 3583)	0.16941894714909375
  (0, 20729)	0.2738578969459924
  (0, 16614)	0.15696073877453268
  (0, 14550)	0.2236809363648068
  (0, 23267)	0.16128139780622516
  (0, 26547)	0.19638671147741732
  (0, 22464)	0.20207089568711745
  (0, 21487)	0.21550128478931546
  (0, 27540)	0.19771357974524176
  (0, 27182)	0.23480088356130552
  (0, 21521)	0.1562356665943368
  (0, 18558)	0.2612168335104853
  (0, 18164)	0.08690831799482265
  (0, 27161)	0.1261988657937151
  (0, 24197)	0.07518543993419265
  (0, 22778)	0.2738578969459924
  (0, 4676)	0.2452909770424961
  (0, 26334)	0.13059723071916296
  (0, 22916)	0.33668756406929184
  (0, 10193)	0.16532432420812987
  (0, 4456)	0.21799675215510664
  (0, 9304)	0.2738578969459924
  (0, 21836)	0.09966592997173944
  (0, 8936)	0.11806131645084653
  (0, 444)	0.09109355212252294
  :	:
  (4801, 5449)	0.31510541707008866
  (4801, 22170)	0.31510541707008866
  (4801, 27361)	0.31510541707008866
  (4801, 27513)	0.3005604008026175
  (4801, 8121)	0.2611505149930138
  (4801,

In [48]:
# getting the similarity score using cosine similarity

similarity = cosine_similarity(feature_vectors)
print(similarity)
print(similarity.shape)

[[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]
 [0.06865296 1.         0.02799128 ... 0.01243107 0.         0.        ]
 [0.01492221 0.02799128 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01243107 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
(4803, 4803)


In [49]:
# getting the movie name from the user

movies_name= input('Enter your favourite movie name:' )

In [50]:
# creating a list of all the movie names given in the dataset 

list_of_all_titles= movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [51]:
# finding the close match for the movie name given by the user

close_match = difflib.get_close_matches(movies_name, list_of_all_titles)
print(close_match)

['Iron Man 2', 'Iron Man 3', 'Iron Man']


In [52]:
closest_match= close_match[0]
print(closest_match)

Iron Man 2


In [53]:
# finding the index of the movie using the title

index_of_movie= movies_data[movies_data['title'] == closest_match]['index'].values[0]
print(index_of_movie)

79


In [55]:
# getting a list of similar movies

similarity_score= list(enumerate(similarity[index_of_movie]))
print(similarity_score)
print(len(similarity_score))

[(0, 0.037286007139398114), (1, 0.05317132679631266), (2, 0.038960496143137506), (3, 0.0061000723807452415), (4, 0.03077907500571099), (5, 0.0057191355374116735), (6, 0.007700520725078716), (7, 0.3259056322221998), (8, 0.007889857417743297), (9, 0.1587802851989625), (10, 0.043078505687390835), (11, 0.012107907694529497), (12, 0.007630316787101211), (13, 0.027450986872019335), (14, 0.22084453253307146), (15, 0.01635788799935997), (16, 0.3079262376488009), (17, 0.012838401161227928), (18, 0.02900291986552577), (19, 0.03826078799304132), (20, 0.09992107373576029), (21, 0.004972916752882749), (22, 0.02178752451385639), (23, 0.032834118723787886), (24, 0.021381081086659707), (25, 0.009600513565275687), (26, 0.2662261136150401), (27, 0.01990093898049068), (28, 0.021421034575843496), (29, 0.023790439501106646), (30, 0.12731940485635315), (31, 0.3582607046317052), (32, 0.00868351768314761), (33, 0.22559079179439678), (34, 0.0), (35, 0.015535680768441574), (36, 0.033428852826899315), (37, 0.007

In [56]:
# sorting the movies based on their similarity score

sorted_similar_movies= sorted(similarity_score, key= lambda x:x[1], reverse=True)
print(sorted_similar_movies)

[(79, 1.0), (31, 0.3582607046317052), (7, 0.3259056322221998), (16, 0.3079262376488009), (511, 0.26943522389964164), (26, 0.2662261136150401), (788, 0.24123346124568507), (68, 0.23663084372818138), (203, 0.23444052754347716), (64, 0.23108400711603602), (182, 0.22721937935631015), (33, 0.22559079179439678), (14, 0.22084453253307146), (126, 0.2185696028149635), (38, 0.21823540153772839), (46, 0.2108242140818431), (85, 0.21019343462787318), (174, 0.1934238551963935), (4759, 0.1757248482082524), (870, 0.16473786456420714), (9, 0.1587802851989625), (782, 0.15709371426357915), (1177, 0.15505122243378922), (101, 0.14809899232311566), (1192, 0.1415976502361903), (1740, 0.13296091475836574), (1977, 0.13007298497462497), (2235, 0.12744852635948345), (30, 0.12731940485635315), (241, 0.12632004675986877), (1428, 0.12497173880477429), (129, 0.12318079361728398), (1720, 0.12126766964808673), (1445, 0.11964244955833338), (700, 0.11889746747207604), (2487, 0.11396935911523207), (668, 0.113637051953228

In [57]:
print('Movies suggested for you: \n')
i=1
for movies in sorted_similar_movies:
    index = movies[0]
    title_from_index = movies_data[movies_data['index']==index]['title'].values[0]

    if(i<30):
        print(i, '.', title_from_index)
        i+=1

Movies suggested for you: 

1 . Iron Man 2
2 . Iron Man 3
3 . Avengers: Age of Ultron
4 . The Avengers
5 . X-Men
6 . Captain America: Civil War
7 . Deadpool
8 . Iron Man
9 . X2
10 . X-Men: Apocalypse
11 . Ant-Man
12 . X-Men: The Last Stand
13 . Man of Steel
14 . Thor: The Dark World
15 . The Amazing Spider-Man 2
16 . X-Men: Days of Future Past
17 . Captain America: The Winter Soldier
18 . The Incredible Hulk
19 . The Image Revolution
20 . Superman II
21 . Batman v Superman: Dawn of Justice
22 . The Spirit
23 . Sin City
24 . X-Men: First Class
25 . Spawn
26 . Kick-Ass 2
27 . The Wiz
28 . A Scanner Darkly
29 . Spider-Man 2
